<a href="https://colab.research.google.com/github/Gizelle-16/Gitel_Scifor/blob/main/Twitter_data_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installing dataset via kaggle through API

In [1]:
! pip install kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d kazanova/sentiment140

 98% 79.0M/80.9M [00:03<00:00, 25.8MB/s]
100% 80.9M/80.9M [00:03<00:00, 23.0MB/s]


Extracting the data from zip file

In [4]:
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('extracted')


extracted


importing libraries

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


Data pre-processing

In [7]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding ='ISO-8859 -1')
df.shape
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
column_names =['target', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding ='ISO-8859 -1')
df.shape
df.head()


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

converting 4s to 1s in target

In [10]:
df.replace({'target':{4:1}}, inplace = True) #conversion happens in actal dataset and not in copy
df['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

stemming to convert word into root word.

In [11]:
port_stem = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join (stemmed_content)
  return stemmed_content

df['stemmed_content']= df['text'].apply(stemming)




In [12]:
print(df['stemmed_content'])



0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [13]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [14]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [15]:
X = df['stemmed_content'].values
Y = df['target'].values



In [16]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [17]:
print(Y)


[0 0 0 ... 1 1 1]


In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state = 5)
#stratify to get equal share of Y i.e 0 and 1s
#random state defines how to split



In [19]:
print(X.shape, X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [20]:
print(X_test)

['emili like pee deep end ahhaah'
 'mean see aaaaaaaaand way dailybooth meet best friend http tinyurl com mwsm w'
 'go ring shop ladi' ... 'nfrussel yeah road trip'
 'ptinsley damn suck load librari yet test music command ruin day'
 'mileycyru new song come along good hope hope well famili god bless chanc repli']


In [21]:

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [22]:
print(X_test)

  (0, 311854)	0.4171144373494487
  (0, 234671)	0.18241073412253994
  (0, 120039)	0.27350243321462087
  (0, 118629)	0.39959613334366184
  (0, 96204)	0.3883553643040905
  (0, 6647)	0.6383113115794222
  (1, 436990)	0.21823309985776707
  (1, 409728)	0.272543871815237
  (1, 358217)	0.18851425508300926
  (1, 259198)	0.25961146603053226
  (1, 258502)	0.2499920169561539
  (1, 171195)	0.1742967177513577
  (1, 138886)	0.21710676644564528
  (1, 88887)	0.4596247711759963
  (1, 78763)	0.1870233790393096
  (1, 39493)	0.2358289629722106
  (1, 98)	0.5773211737315743
  (2, 365683)	0.4956902588423947
  (2, 341209)	0.6190504487881319
  (2, 225636)	0.5431230906491545
  (2, 149798)	0.2758351274723409
  (3, 324595)	0.20184426143574094
  (3, 319447)	0.26381790544732014
  (3, 283716)	0.26568852073370863
  (3, 261875)	0.34469144514205413
  :	:
  (319997, 342468)	0.6622239623228642
  (319998, 454554)	0.221774674250392
  (319998, 399548)	0.26065775606923364
  (319998, 387191)	0.21599396634117232
  (319998, 34740

model training

In [23]:
model = LogisticRegression(max_iter = 1000)


In [24]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

accuracy score

In [25]:
#training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(  Y_train, X_train_prediction)
print('the accuracy is:', training_data_accuracy)

the accuracy is: 0.810353125


In [26]:
#test data accuracy
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(  Y_test, X_test_prediction)
print('the accuracy is:', test_data_accuracy)


the accuracy is: 0.777621875


since we can see overfitting, lets perform feature extraction once again

In [27]:
from sklearn.metrics import f1_score
training_f1 = f1_score(Y_train, X_train_prediction, average = 'binary')
test_f1 = f1_score(Y_test, X_test_prediction, average = 'binary')
print(training_f1, test_f1)

0.8141542513283008 0.7819093998915079


In [28]:
from sklearn.model_selection import cross_val_score


In [29]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [30]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)



In [31]:
# Perform cross-validation and get accuracy scores on training data
cv_scores = cross_val_score(model, X_train, Y_train, cv=k_fold, scoring='accuracy')
# Print average accuracy across all folds
print("Average Accuracy (Cross-Validation):", np.mean(cv_scores))



Average Accuracy (Cross-Validation): 0.7771273437499999


In [32]:

# Step 6: Fit the final model on the entire training set after cross-validation
model.fit(X_train, Y_train)



LogisticRegression(max_iter=1000)

In [34]:
# Step 7: Evaluate on the test set
Y_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, Y_test_prediction)
print('Test Data Accuracy:', test_data_accuracy)

Test Data Accuracy: 0.777621875


In [35]:
import pickle

In [36]:
filename = 'trained_modelSentimental.sav'
pickle.dump(model, open(filename, 'wb'))

In [37]:
#loading the saved model0
loaded_model = pickle.load(open('/content/trained_modelSentimental.sav', 'rb'))

cheking with random data point

In [45]:
x_new = X_test[200]
print(Y_test[200])
prediction = model.predict(x_new)

print(prediction)

if prediction[0] == 0:
  print('negative')
else:
  print('positive')

1
[1]
positive


In [46]:
user_input = input("Enter a string for sentiment analysis: ")

user_input_tfidf = vectorizer.transform([user_input])
prediction = model.predict(user_input_tfidf)

if prediction[0] == 1:
  print('this is a positive tweet')
else:
  print('negative emotion')

Enter a string for sentiment analysis: bad day today
negative emotion
